In [1]:
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
def preprocess_keypoints_data(json_folder, label_map, output_file):
    """
    Preprocess keypoints data from JSON files for model training.

    Args:
        json_folder (str): Path to the folder containing keypoints JSON files.
        label_map (dict): Mapping of exercise names to numerical labels.
        output_file (str): Path to save the preprocessed data.

    Returns:
        None
    """
    data = []
    labels = []

    for exercise_folder in os.listdir(json_folder):
        exercise_path = os.path.join(json_folder, exercise_folder)

        if os.path.isdir(exercise_path):
            print(f"Processing exercise folder: {exercise_folder}")

            label = label_map.get(exercise_folder, None)
            if label is None:
                print(f"Warning: No label found for {exercise_folder}. Skipping...")
                continue

            for json_file in os.listdir(exercise_path):
                if json_file.endswith(".json"):
                    json_path = os.path.join(exercise_path, json_file)

                    with open(json_path, "r") as f:
                        keypoints_data = json.load(f)

                    normalized_keypoints = []
                    for frame in keypoints_data["keypoints"]:
                        frame_keypoints = []
                        for point in frame["keypoints"]:
                            x = point["x"]
                            y = point["y"]
                            z = point["z"]
                            visibility = point["visibility"]

                            frame_keypoints.extend([x, y, z, visibility])

                        normalized_keypoints.append(frame_keypoints)

                    data.append(normalized_keypoints)
                    labels.append(label)

    data = np.array(data, dtype=object)
    labels = np.array(labels)

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)
    val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels, test_size=0.3, random_state=42)

    np.savez_compressed(output_file, train_data=train_data, train_labels=train_labels,
                        val_data=val_data, val_labels=val_labels,
                        test_data=test_data, test_labels=test_labels)
    print(f"Preprocessed data saved to {output_file}")

In [3]:
label_map = {
    "Neck Flexion": 0,
    "Neck Extension": 1,
    "Neck Rotation": 2,
    "Neck Side Bend": 3,
    "Seated Neck Flexion": 4,
    "Seated Neck Stretch": 5,
    "Seated Head Turns": 6,
    "Shoulder Shrugs": 7,
    "Neck Isometrics": 8,
    "Chin Tucks": 9,
}

In [4]:
json_folder = "/content/drive/MyDrive/Exercise Tracking/keypoints_data"
output_file = "/content/drive/MyDrive/Exercise Tracking/preprocessed_data.npz"

preprocess_keypoints_data(json_folder, label_map, output_file)

Processing exercise folder: Neck Isometrics
Processing exercise folder: Seated Neck Flexion
Processing exercise folder: Seated Neck Stretch
Processing exercise folder: Neck Side Bend
Processing exercise folder: Neck Extension
Processing exercise folder: Shoulder Shrugs
Processing exercise folder: Chin Tucks
Processing exercise folder: Seated Head Turns
Processing exercise folder: Neck Flexion
Processing exercise folder: Neck Rotation
Preprocessed data saved to /content/drive/MyDrive/Exercise Tracking/preprocessed_data.npz
